Content Based Recommendation

In [1]:
import numpy as np
import pandas as pd
import sklearn
from ast import literal_eval
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

import difflib

In [2]:
#ratings = pd.read_csv (r'./../data/ratings.csv', low_memory=False)
#ratings_small = pd.read_csv (r'./../data/ratings_small.csv', low_memory=False)
credits = pd.read_csv (r'./../data/credits.csv', low_memory=False)
keywords = pd.read_csv (r'./../data/keywords.csv', low_memory=False)
#links = pd.read_csv (r'./../data/links.csv', low_memory=False)
#links_small = pd.read_csv (r'./../data/links_small.csv', low_memory=False)
metadata = pd.read_csv (r'./../data/movies_metadata.csv', low_memory=False)

In [3]:
#ratings.head()

In [4]:
#credits.head()

In [5]:
#metadata.head()

In [6]:
metadata = metadata[~metadata.id.str.contains("-")]

**Recommendation based on different metadata**  
Actors, director, keywords, etc. are looked at as well

In [7]:
# Convert IDs to int. Required for merging
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
metadata['id'] = metadata['id'].astype('int')

In [8]:
# Merge keywords and credits into your main metadata dataframe
metadata = metadata.merge(credits, on='id')
metadata = metadata.merge(keywords, on='id')

In [9]:
# Parse the stringified features into their corresponding python objects
features = ['cast', 'crew', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(literal_eval)

In [10]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [11]:
# Define new director, cast, genres and keywords features that are in a suitable form.
metadata['director'] = metadata['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    metadata[feature] = metadata[feature].apply(get_list)

In [12]:
# Function to convert all strings to lower case and strip names of spaces
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

# Apply clean_data function to your features.
features = ['cast', 'keywords', 'director', 'genres']

for feature in features:
    metadata[feature] = metadata[feature].apply(clean_data)

In [13]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])

# Create a new soup feature
metadata['soup'] = metadata.apply(create_soup, axis=1)

metadata[['soup']].head(2)

,soup
0,jealousy toy boy tomhanks timallen donrickles ...
1,boardgame disappearance basedonchildren'sbook ...


In [14]:
# create count matrix with CountVectorizer
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(metadata['soup'])
count_matrix.shape

(46628, 73881)

In [15]:
# Compute the Cosine Similarity matrix based on the count_matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [22]:
pd.DataFrame(cosine_sim).to_csv(r'..\cosine_sim2.csv')

KeyboardInterrupt: 

In [67]:
# Reset index of your main DataFrame and construct reverse mapping as before
metadata = metadata.reset_index()
indices = pd.Series(metadata.index, index=metadata['id'])

In [68]:
def get_recommendations(id, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[id]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return metadata['title'].iloc[movie_indices]

In [98]:
input_title = input("Your title: ")

Your title: double trouble


In [99]:
possible_titles = difflib.get_close_matches(input_title, metadata['original_title'].tolist(), 5)
print(possible_titles)

['Double Trouble', 'Double Trouble', 'Triple Trouble', 'Triple Trouble', 'Monkey Trouble']


In [100]:
id_array = []
for i in possible_titles:
    id_array.append(metadata.loc[metadata['original_title'] == str(i), 'id'].array[0])

In [101]:
print(id_array)

[45335, 45335, 184328, 184328, 41582]


In [102]:
input_number = int(input("What's your film? "))
chosen_id = id_array[input_number]

What's your film? 0


In [103]:
print(chosen_id)

45335


In [104]:
get_recommendations(chosen_id, cosine_sim)

3505                         Blue Hawaii
8174     It Happened at the World's Fair
5961                Girls! Girls! Girls!
3507                          G.I. Blues
8176                             Spinout
9911          The Boys from County Clare
6184           Down & Out With The Dolls
9909                          Girl Crazy
18630                What's Up, Scarlet?
23438                Made For Each Other
Name: title, dtype: object